# Learn about the optimization loop
## Optimizing the model parameters
- 모델을 학습하는 것은 반복적인 프로세스이며 epoch로 불리우는 각 반복 단계에서 모델은 출력과 에러를 계산
- 이후에는 모듈에서의 파라미터와 관련된 에러의 미분값을 수집하고 경사하강법을 사용하여 파라미터를 최적화함
- 이 프로세스의 자세한 내용은 [backpropagation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8)를 참고바람

### Prerequisite code
앞에서 소개된 코드로부터 **Dataset**, **DataLoaders**, **Build Model** 모듈이 필요

In [47]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [48]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

## Setting hyperparameters
하이퍼파라미터는 모델 최적화 프로세스를 제어하는 조절 가능한 파라미터를 의미함. 하이퍼파라미터는 모델 학습과 수렴 속도에 영향을 줌([참고](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html))

모델 훈련을 위해 다음의 하이퍼파라미터를 정의
- **Number of Epochs** - 데이터 전체의 반복 횟수
- **Bactch Size** - 각 epoch에서 모델에 보여지는 데이터 샘플 수 
- **Learning Rate** - 각 batch와 epoch에서의 업데이트 크기로 작은 값는 학습 속도가 느려지지만, 큰 값은 학습 동안 예측할 수 없는 결과를 초래함

In [49]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Add an optimization loop

하이퍼파라미터를 설정한 다음으로 최적화 반복을 사용하여 모델 학습과 최적화할 수 있음. 최적화를 위한 각 반복을 **epoch**라 부름. 각 epoch는 두 부분으로 구성됨.
- The Train Loop - 데이터 전체를 학습하고 최적 파라미터로 수렴을 반복적으로 시도함
- The validation/Test Loop - 테스트 데이터 전반에서 모델 성능이 향상되었는지 확인을 반복함

### Add a loss function
손실함수는 목표값과 모델로 부터 얻은 값 사이의 차이를 측정하므로 학습하는 동안 최소화되길 원함. 손실을 계산하기 위해 주어진 데이터 샘플을 사용하여 예측한 값과 실제  값의 차이를 비교함

일반적인 손실 함수는 [nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss) (Mean Square Error, regression), [nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss) (Negative Log Likelihood, classification), [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) (combines `nn.LogSoftmax` and `nn.NLLLoss`.)

출력 logit을 `nn.CrossEntropyLoss`로 전달하면, logit을 정규화하고 및 예측 에러를 계산함

In [50]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimization pass
- 최적화는 각 학습 단계에서 모델 오차를 줄이기 위해 파라미터를 조절하는 단계
- 모든 최적화 로직은 `optimizer` object로 캡슐레이션 되어있음
- ADAM, RMSProp와 같은 다양한 최적화 알고리즘을 사용할 수 있음 ([different optimizers](https://pytorch.org/docs/stable/optim.html))
- 학습률를 전달하고 학습에 필요한 파라미터를 등록함으로써 optimizer를 초기화 할 수 있음

In [51]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

반복 학습과정 내부에는 최적화가 3단계로 진행됨
- 모델 파라미터의 경사 초기화를 위해 `optimizer.zero_grad()` 호출함. 기본값에 의해 경사는 누적됨. 이중 계산을 막기 위해 각 반복에서 명시적으로 0을 초기화해야 함
- `loss.backwards()`을 사용하여 예측 오차를 역전파. Pytorch는 각 파라미터 손실의 경사를 저장함.
- `optimizer.step()` 호출하면 역전파 경로에서 수집된 경사로 인해 파라미터를 조절할 수 있음.


### Full implementation

In [52]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

`train_loop` 와 `test_loop`에 손실함수, 최적화 함수를 전달. 모델 성능향상을 위해 epoch 수를 높게 조절할 수 있음.

In [53]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300640  [    0/60000]
loss: 2.290309  [ 6400/60000]
loss: 2.286352  [12800/60000]
loss: 2.284961  [19200/60000]
loss: 2.274725  [25600/60000]
loss: 2.273580  [32000/60000]
loss: 2.268600  [38400/60000]
loss: 2.267196  [44800/60000]
loss: 2.237566  [51200/60000]
loss: 2.240211  [57600/60000]
Test Error: 
 Accuracy: 43.2%, Avg loss: 0.035035 

Epoch 2
-------------------------------
loss: 2.225451  [    0/60000]
loss: 2.211274  [ 6400/60000]
loss: 2.202846  [12800/60000]
loss: 2.202437  [19200/60000]
loss: 2.185632  [25600/60000]
loss: 2.186528  [32000/60000]
loss: 2.178825  [38400/60000]
loss: 2.179127  [44800/60000]
loss: 2.110761  [51200/60000]
loss: 2.115404  [57600/60000]
Test Error: 
 Accuracy: 43.8%, Avg loss: 0.033081 

Epoch 3
-------------------------------
loss: 2.113491  [    0/60000]
loss: 2.087825  [ 6400/60000]
loss: 2.064968  [12800/60000]
loss: 2.056619  [19200/60000]
loss: 2.046947  [25600/60000]
loss: 2.065896  [32000/600